---
title: "TF-IDF와 BM25 구현"
date: "2025-11-10"
category: "Information Retrieval"
tags: ["IR"]
excerpt: "TF-IDF, BM25랑 역색인 구현하기"
---

In [4]:
docs = {
    "D1": "I love machine learning",
    "D2": "machine learning is powerful",
    "D3": "I love deep learning"
}

In [5]:
def build_inverted_index(docs):
    """
    역색인(inverted index)을 구축합니다.
    
    Args:
        docs: 문서 딕셔너리 {doc_id: text}
    
    Returns:
        역색인 딕셔너리 {term: [doc_ids]}
    """
    inverted_index = {}
    
    for doc_id, text in docs.items():
        # 텍스트를 소문자로 변환하고 단어로 분리
        terms = text.lower().split()
        
        for term in terms:
            if term not in inverted_index:
                inverted_index[term] = []
            
            # 문서 ID가 이미 리스트에 없으면 추가
            if doc_id not in inverted_index[term]:
                inverted_index[term].append(doc_id)
    
    return inverted_index

# 역색인 구축
inverted_index = build_inverted_index(docs)
inverted_index

{'i': ['D1', 'D3'],
 'love': ['D1', 'D3'],
 'machine': ['D1', 'D2'],
 'learning': ['D1', 'D2', 'D3'],
 'is': ['D2'],
 'powerful': ['D2'],
 'deep': ['D3']}

In [6]:
term = 'love'

terms = docs['D1'].lower().split()
terms.count(term.lower())

1

In [7]:
import math
import pandas as pd

def term_frequency(term: str, doc_text: str) -> int:
    """
    Term Frequency (TF)를 계산합니다.
    문서 내에서 특정 단어가 나타나는 빈도입니다.
    
    Args:
        term: 단어
        doc_text: 문서 텍스트
    
    Returns:
        TF 값 (단어 빈도)
    """
    terms = doc_text.lower().split()
    return terms.count(term.lower())

def inverse_document_frequency(term: str, docs: dict, inverted_index: dict) -> float:
    """
    Inverse Document Frequency (IDF)를 계산합니다.
    전체 문서 집합에서 특정 단어가 나타나는 문서의 역수를 로그로 계산합니다.

    IDF 공식:
        IDF(term) = log(N / df)
        - N: 전체 문서 수 (total_docs)
        - df: 해당 단어를 포함하는 문서 수 (doc_frequency)
    
    Args:
        term: 단어
        docs: 문서 딕셔너리
        inverted_index: 역색인
    
    Returns:
        IDF 값
    """
    total_docs = len(docs)
    
    # 역색인에서 해당 단어가 나타나는 문서 수(df)
    if term.lower() in inverted_index:
        doc_frequency = len(inverted_index[term.lower()])
    else:
        doc_frequency = 0
    
    # IDF 계산: log(전체 문서 수 / 해당 단어를 포함하는 문서 수)
    # 공식: IDF(term) = log(N / df)
    # 분모(df)가 0이 되는 것을 방지하기 위해 0이면 0 반환
    if doc_frequency == 0:
        return 0
    
    return math.log(total_docs / doc_frequency)

def tf_idf(term: str, doc_id: str, docs: dict, inverted_index: dict) -> float:
    """
    TF-IDF 점수를 계산합니다.
    TF * IDF
    
    Args:
        term: 단어
        doc_id: 문서 ID
        docs: 문서 딕셔너리
        inverted_index: 역색인
    
    Returns:
        TF-IDF 점수
    """
    tf = term_frequency(term, docs[doc_id])
    idf = inverse_document_frequency(term, docs, inverted_index)
    return tf * idf

def build_vocabulary(docs: dict) -> list:
    """
    전체 문서 집합에서 단어 사전(vocabulary)을 구축합니다.
    
    Args:
        docs: 문서 딕셔너리
    
    Returns:
        정렬된 단어 리스트 (전체 vocabulary)
    """
    vocabulary = set()
    for doc_text in docs.values():
        terms = doc_text.lower().split()
        vocabulary.update(terms)
    return sorted(vocabulary)

def compute_tf_idf_scores(
    docs: dict,              # 문서 딕셔너리 {doc_id: doc_text}
    inverted_index: dict,    # 역색인 {term: [doc_id, ...]}
    vocabulary: list         # 전체 단어 사전 (정렬된 단어 리스트)
) -> dict:
    """
    모든 문서에 대한 TF-IDF 벡터를 계산합니다.
    전체 vocabulary에 맞춰 벡터를 생성합니다.
    
    Args:
        docs: 문서 딕셔너리
        inverted_index: 역색인
        vocabulary: 전체 단어 사전
    
    Returns:
        TF-IDF 벡터 딕셔너리 {doc_id: {term: tf_idf_score}}
        모든 문서 벡터는 동일한 vocabulary 차원을 가집니다.
    """
    tf_idf_scores = {}
    
    # 모든 문서에 대해
    for doc_id in docs.keys():
        tf_idf_scores[doc_id] = {}
        
        # 전체 vocabulary의 각 단어에 대해
        for term in vocabulary:
            # 해당 문서에 단어가 있으면 TF-IDF 계산, 없으면 0
            tf_idf_scores[doc_id][term] = tf_idf(term, doc_id, docs, inverted_index)
    
    return tf_idf_scores

# 전체 단어 사전 구축
vocabulary = build_vocabulary(docs)
print(f"전체 단어 사전 (Vocabulary): {vocabulary}")
print(f"차원 수: {len(vocabulary)}\n")

# TF-IDF 벡터 계산 (전체 vocabulary에 맞춰)
tf_idf_scores = compute_tf_idf_scores(docs, inverted_index, vocabulary)

# DataFrame으로 변환
tf_idf_df = pd.DataFrame(
    [[tf_idf_scores[doc_id][term] for term in vocabulary] 
     for doc_id in sorted(docs.keys())],
    columns=vocabulary,
    index=sorted(docs.keys())
)

print("TF-IDF 벡터 (DataFrame):")
tf_idf_df

전체 단어 사전 (Vocabulary): ['deep', 'i', 'is', 'learning', 'love', 'machine', 'powerful']
차원 수: 7

TF-IDF 벡터 (DataFrame):


,deep,i,is,learning,love,machine,powerful
D1,0.000000,0.405465,0.000000,0.0,0.405465,0.405465,0.000000
D2,0.000000,0.000000,1.098612,0.0,0.000000,0.405465,1.098612
D3,1.098612,0.405465,0.000000,0.0,0.405465,0.000000,0.000000


1. query도 하나의 문서라고 생각하고 tf-idf vector 계산
2. 이걸 사용해서 cosine similarity 계산

In [8]:
def query_tf_idf(
    query: str,
    docs: dict,
    inverted_index: dict,
    vocabulary: list
) -> dict:
    """
    쿼리에 대한 TF-IDF 벡터를 계산합니다.
    전체 vocabulary에 맞춰 벡터를 생성합니다.
    
    Args:
        query: 쿼리 문자열
        docs: 문서 딕셔너리
        inverted_index: 역색인
        vocabulary: 전체 단어 사전
    
    Returns:
        쿼리 TF-IDF 벡터 {term: tf_idf_score}
        전체 vocabulary 차원을 가집니다.
    """
    query_terms = query.lower().split()
    query_tf_idf_vector = {}
    
    # 전체 vocabulary의 각 단어에 대해
    for term in vocabulary:
        if term in query_terms:
            # 쿼리 내에서의 TF (빈도)
            tf = query_terms.count(term)
            # 문서 집합에서의 IDF
            idf = inverse_document_frequency(term, docs, inverted_index)
            query_tf_idf_vector[term] = tf * idf
        else:
            # 쿼리에 없는 단어는 0
            query_tf_idf_vector[term] = 0
    
    return query_tf_idf_vector

def cosine_similarity(
    vec1: dict, 
    vec2: dict, 
    vocabulary: list
) -> float:
    """
    두 벡터 간의 코사인 유사도를 계산합니다.
    두 벡터는 동일한 vocabulary 차원을 가집니다.
    
    Args:
        vec1: 첫 번째 벡터 (딕셔너리, vocabulary의 모든 단어 포함)
        vec2: 두 번째 벡터 (딕셔너리, vocabulary의 모든 단어 포함)
        vocabulary: 전체 단어 사전
    
    Returns:
        코사인 유사도 (0~1)
    """
    # 벡터의 내적 계산 (동일한 차원에서)
    dot_product = sum(vec1[term] * vec2[term] for term in vocabulary)
    
    # 벡터의 크기 계산
    magnitude1 = math.sqrt(sum(vec1[term] ** 2 for term in vocabulary))
    magnitude2 = math.sqrt(sum(vec2[term] ** 2 for term in vocabulary))
    
    # 코사인 유사도
    if magnitude1 == 0 or magnitude2 == 0:
        return 0
    
    return dot_product / (magnitude1 * magnitude2)

def search_documents(
    query: str,
    docs: dict,
    inverted_index: dict,
    tf_idf_scores: dict,
    vocabulary: list
) -> list:
    """
    쿼리에 대해 가장 관련성 높은 문서를 검색합니다.
    
    Args:
        query: 검색 쿼리 문자열
        docs: 문서 딕셔너리
        inverted_index: 역색인
        tf_idf_scores: 문서들의 TF-IDF 벡터
        vocabulary: 전체 단어 사전
    
    Returns:
        (doc_id, similarity_score) 튜플의 리스트, 유사도 순으로 정렬
    """
    # 쿼리 TF-IDF 벡터 계산 (전체 vocabulary에 맞춰)
    query_vector = query_tf_idf(query, docs, inverted_index, vocabulary)
    
    # 각 문서와의 유사도 계산
    similarities = []
    for doc_id in docs.keys():
        doc_vector = tf_idf_scores[doc_id]
        similarity = cosine_similarity(query_vector, doc_vector, vocabulary)
        similarities.append((doc_id, similarity))
    
    # 유사도 순으로 정렬 (내림차순)
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return similarities

# "I love you" 쿼리로 검색
query = "I love you"
results = search_documents(query, docs, inverted_index, tf_idf_scores, vocabulary)

print(f"쿼리: '{query}'")
print("\n검색 결과 (유사도 순):")
for doc_id, similarity in results:
    print(f"  {doc_id}: {similarity:.4f} - {docs[doc_id]}")

print("\n쿼리 TF-IDF 벡터 (전체 vocabulary 기준):")
query_vector = query_tf_idf(query, docs, inverted_index, vocabulary)
query_vector

쿼리: 'I love you'

검색 결과 (유사도 순):
  D1: 0.8165 - I love machine learning
  D3: 0.4627 - I love deep learning
  D2: 0.0000 - machine learning is powerful

쿼리 TF-IDF 벡터 (전체 vocabulary 기준):


{'deep': 0,
 'i': 0.4054651081081644,
 'is': 0,
 'learning': 0,
 'love': 0.4054651081081644,
 'machine': 0,
 'powerful': 0}

# BM25 구현

BM25는 TF-IDF의 한계를 개선한 ranking function입니다.

**주요 개선점:**
1. **문서 길이 정규화**: `b` 파라미터로 긴 문서의 불이익 방지
2. **TF 포화 효과**: `k₁` 파라미터로 단어 반복의 비선형 증가

**BM25 공식:**

$$
\text{BM25}(t, d) = \text{IDF}(t) \cdot \frac{f(t, d) \cdot (k_1 + 1)}{f(t, d) + k_1 \cdot (1 - b + b \cdot \frac{|d|}{\text{avgdl}})}
$$

여기서:
- $f(t, d)$: 문서 $d$에서 단어 $t$의 빈도 (TF)
- $|d|$: 문서 $d$의 길이 (단어 개수)
- $\text{avgdl}$: 전체 문서의 평균 길이
- $k_1$: TF 포화를 조절하는 파라미터 (보통 1.2~2.0, 기본값 1.5) -> 작을수록 포화가 더 빨리됨. 억제효과가 커짐
- $b$: 문서 길이 정규화 파라미터 (0~1, 기본값 0.75) -> 커질수록 긴 문서에 높은 패널티
- $\text{IDF}(t) = \log\left(\frac{N - df(t) + 0.5}{df(t) + 0.5} + 1\right)$

In [17]:
import math

def bm25_idf(term: str, docs: dict, inverted_index: dict) -> float:
    """
    BM25의 IDF를 계산합니다.
    
    IDF(t) = log((N - df(t) + 0.5) / (df(t) + 0.5) + 1)
    
    Args:
        term: 단어
        docs: 문서 딕셔너리
        inverted_index: 역색인
    
    Returns:
        BM25 IDF 값
    """
    N = len(docs)  # 전체 문서 수
    
    # 해당 단어를 포함하는 문서 수
    if term.lower() in inverted_index:
        df = len(inverted_index[term.lower()])
    else:
        df = 0
    
    # BM25 IDF 공식
    idf = math.log((N - df + 0.5) / (df + 0.5) + 1)
    return idf


def document_length(doc_text: str) -> int:
    """
    문서의 길이(단어 개수)를 계산합니다.
    
    Args:
        doc_text: 문서 텍스트
    
    Returns:
        문서 길이 (단어 개수)
    """
    return len(doc_text.lower().split())


def average_document_length(docs: dict) -> float:
    """
    전체 문서의 평균 길이를 계산합니다.
    
    Args:
        docs: 문서 딕셔너리
    
    Returns:
        평균 문서 길이
    """
    total_length = sum(document_length(doc_text) for doc_text in docs.values())
    return total_length / len(docs)


def bm25_score(
    term: str,
    doc_id: str,
    docs: dict,
    inverted_index: dict,
    avgdl: float,
    k1: float = 1.5,
    b: float = 0.75
) -> float:
    """
    특정 단어에 대한 BM25 점수를 계산합니다.
    
    BM25(t, d) = IDF(t) * (f(t,d) * (k1 + 1)) / (f(t,d) + k1 * (1 - b + b * |d|/avgdl))
    
    Args:
        term: 단어
        doc_id: 문서 ID
        docs: 문서 딕셔너리
        inverted_index: 역색인
        avgdl: 평균 문서 길이
        k1: TF 포화 파라미터 (기본값 1.5)
        b: 문서 길이 정규화 파라미터 (기본값 0.75)
    
    Returns:
        BM25 점수
    """
    # IDF 계산
    idf = bm25_idf(term, docs, inverted_index)
    
    # TF 계산 (문서에서 단어 빈도)
    tf = term_frequency(term, docs[doc_id])
    
    # 문서 길이
    doc_len = document_length(docs[doc_id])
    
    # BM25 공식
    numerator = tf * (k1 + 1)
    denominator = tf + k1 * (1 - b + b * (doc_len / avgdl))
    
    bm25 = idf * (numerator / denominator)
    
    return bm25


def bm25_search(
    query: str,
    docs: dict,
    inverted_index: dict,
    k1: float = 1.5,
    b: float = 0.75
) -> list:
    """
    BM25를 사용하여 문서를 검색합니다.
    
    Args:
        query: 검색 쿼리
        docs: 문서 딕셔너리
        inverted_index: 역색인
        k1: TF 포화 파라미터
        b: 문서 길이 정규화 파라미터
    
    Returns:
        (doc_id, bm25_score) 튜플의 리스트, 점수 순으로 정렬
    """
    # 평균 문서 길이 계산
    avgdl = average_document_length(docs)
    
    # 쿼리 단어들
    query_terms = query.lower().split()
    
    # 각 문서에 대한 BM25 점수 계산
    scores = {}
    for doc_id in docs.keys():
        score = 0
        for term in query_terms:
            score += bm25_score(term, doc_id, docs, inverted_index, avgdl, k1, b)
        scores[doc_id] = score
    
    # 점수 순으로 정렬
    results = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    return results


In [11]:
# BM25로 검색 실행
query = "I love machine learning"

results = bm25_search(query, docs, inverted_index)

print(f"쿼리: '{query}'")
print(f"\n평균 문서 길이: {average_document_length(docs):.2f}")
print("\n검색 결과 (BM25 점수 순):")
for doc_id, score in results:
    print(f"  {doc_id}: {score:.4f} - {docs[doc_id]}")
    
print("\n\n각 문서별 상세 점수:")
avgdl = average_document_length(docs)
query_terms = query.lower().split()

for doc_id in docs.keys():
    print(f"\n{doc_id}: {docs[doc_id]}")
    print(f"  문서 길이: {document_length(docs[doc_id])}")
    total_score = 0
    for term in query_terms:
        score = bm25_score(term, doc_id, docs, inverted_index, avgdl)
        if score > 0:
            print(f"    '{term}': {score:.4f}")
        total_score += score
    print(f"  총 BM25 점수: {total_score:.4f}")


쿼리: 'I love machine learning'

평균 문서 길이: 4.00

검색 결과 (BM25 점수 순):
  D1: 1.5435 - I love machine learning
  D3: 1.0735 - I love deep learning
  D2: 0.6035 - machine learning is powerful


각 문서별 상세 점수:

D1: I love machine learning
  문서 길이: 4
    'i': 0.4700
    'love': 0.4700
    'machine': 0.4700
    'learning': 0.1335
  총 BM25 점수: 1.5435

D2: machine learning is powerful
  문서 길이: 4
    'machine': 0.4700
    'learning': 0.1335
  총 BM25 점수: 0.6035

D3: I love deep learning
  문서 길이: 4
    'i': 0.4700
    'love': 0.4700
    'learning': 0.1335
  총 BM25 점수: 1.0735


## TF-IDF vs BM25 비교

같은 쿼리로 두 방법을 비교해보겠습니다.


In [13]:
import pandas as pd

# 비교를 위한 데이터 수집
query = "machine learning"

# TF-IDF 검색
tfidf_results = search_documents(query, docs, inverted_index, tf_idf_scores, vocabulary)

# BM25 검색
bm25_results = bm25_search(query, docs, inverted_index)

# 결과를 데이터프레임으로 정리
comparison_data = []
for i, doc_id in enumerate(docs.keys()):
    tfidf_score = next((score for did, score in tfidf_results if did == doc_id), 0)
    bm25_score = next((score for did, score in bm25_results if did == doc_id), 0)
    
    comparison_data.append({
        'Document': doc_id,
        'Text': docs[doc_id],
        'TF-IDF Score': tfidf_score,
        'BM25 Score': bm25_score,
        'TF-IDF Rank': next((idx+1 for idx, (did, _) in enumerate(tfidf_results) if did == doc_id), 0),
        'BM25 Rank': next((idx+1 for idx, (did, _) in enumerate(bm25_results) if did == doc_id), 0)
    })

df_comparison = pd.DataFrame(comparison_data)
print(f"쿼리: '{query}'\n")
print(df_comparison.to_string(index=False))

쿼리: 'machine learning'

Document                         Text  TF-IDF Score  BM25 Score  TF-IDF Rank  BM25 Rank
      D1      I love machine learning      0.577350    0.603535            1          1
      D2 machine learning is powerful      0.252515    0.603535            2          2
      D3         I love deep learning      0.000000    0.133531            3          3


## k1과 b 파라미터의 영향 확인

BM25의 두 파라미터가 검색 결과에 어떤 영향을 주는지 확인해봅시다.

In [14]:
# k1 파라미터 변화 (TF 포화 효과)
print("k1 파라미터 변화 (b=0.75 고정)")
print("=" * 60)

query = "machine learning"
k1_values = [0.5, 1.0, 1.5, 2.0, 3.0]

for k1 in k1_values:
    results = bm25_search(query, docs, inverted_index, k1=k1, b=0.75)
    print(f"\nk1={k1}:")
    for doc_id, score in results:
        print(f"  {doc_id}: {score:.4f}")


k1 파라미터 변화 (b=0.75 고정)


TypeError: 'float' object is not callable

## 더 큰 문서 집합으로 테스트

문서 길이 차이가 큰 경우 BM25의 장점을 더 명확히 볼 수 있습니다.

In [15]:
# 길이가 다른 문서들
docs_long = {
    "D1": "machine learning",  # 짧은 문서
    "D2": "machine learning is a powerful tool for data analysis",  # 중간 문서
    "D3": "machine learning machine learning machine learning is used in many applications and machine learning continues to grow",  # 긴 문서 (반복 많음)
    "D4": "deep learning neural networks",  # 관련 없는 짧은 문서
}

# 역색인 구축
inverted_index_long = build_inverted_index(docs_long)

# Vocabulary 구축
vocabulary_long = build_vocabulary(docs_long)

# TF-IDF 계산
tf_idf_scores_long = compute_tf_idf_scores(docs_long, inverted_index_long, vocabulary_long)

# 검색 쿼리
query = "machine learning"

print(f"쿼리: '{query}'")
print(f"평균 문서 길이: {average_document_length(docs_long):.2f}\n")

# 각 문서 정보
print("문서 정보:")
for doc_id, text in docs_long.items():
    doc_len = document_length(text)
    ml_count = text.lower().count("machine learning")
    print(f"  {doc_id} (길이={doc_len:2d}, 'machine learning' 출현={ml_count}회): {text[:50]}...")

print("\n" + "="*80)

# TF-IDF 검색
print("\nTF-IDF 검색 결과:")
tfidf_results_long = search_documents(query, docs_long, inverted_index_long, tf_idf_scores_long, vocabulary_long)
for doc_id, score in tfidf_results_long:
    doc_len = document_length(docs_long[doc_id])
    print(f"  {doc_id} (길이={doc_len:2d}): {score:.4f}")

# BM25 검색
print("\nBM25 검색 결과:")
bm25_results_long = bm25_search(query, docs_long, inverted_index_long)
for doc_id, score in bm25_results_long:
    doc_len = document_length(docs_long[doc_id])
    print(f"  {doc_id} (길이={doc_len:2d}): {score:.4f}")


쿼리: 'machine learning'
평균 문서 길이: 8.00

문서 정보:
  D1 (길이= 2, 'machine learning' 출현=1회): machine learning...
  D2 (길이= 9, 'machine learning' 출현=1회): machine learning is a powerful tool for data analy...
  D3 (길이=17, 'machine learning' 출현=4회): machine learning machine learning machine learning...
  D4 (길이= 4, 'machine learning' 출현=0회): deep learning neural networks...


TF-IDF 검색 결과:
  D1 (길이= 2): 1.0000
  D3 (길이=17): 0.2776
  D2 (길이= 9): 0.0827
  D4 (길이= 4): 0.0000

BM25 검색 결과:


TypeError: 'float' object is not callable

In [16]:
# b 파라미터 변화 (문서 길이 정규화)
print("b 파라미터 변화 (k1=1.5 고정)")
print("=" * 60)

query = "machine learning"
b_values = [0.0, 0.25, 0.5, 0.75, 1.0]

for b in b_values:
    results = bm25_search(query, docs, inverted_index, k1=1.5, b=b)
    print(f"\nb={b}:")
    for doc_id, score in results:
        doc_len = document_length(docs[doc_id])
        print(f"  {doc_id} (길이={doc_len}): {score:.4f}")


b 파라미터 변화 (k1=1.5 고정)


TypeError: 'float' object is not callable

## 핵심 정리

**TF-IDF의 문제점:**
1. D3처럼 "machine learning"이 4번 반복되는 긴 문서가 TF-IDF에서 가장 높은 점수를 받음
2. 문서 길이를 고려하지 않아 단순 반복이 유리함

**BM25의 개선:**
1. **TF 포화 효과**: 단어가 여러 번 반복되어도 점수 증가가 완만해짐
2. **문서 길이 정규화**: 긴 문서에 패널티를 줘서 공정한 비교 가능
3. 결과적으로 D1(짧고 정확한 문서)이나 D2(적절한 길이)가 더 높은 순위를 받음

**파라미터 설정:**
- `k₁`: 1.2~2.0 (기본값 1.5) - 높을수록 TF의 영향이 커짐
- `b`: 0~1 (기본값 0.75) - 높을수록 문서 길이 정규화가 강해짐
